In [ ]:
# Author: Naveen Lalwani
# Script to distill knowledge from LeNet-5 model trained on CIFAR-10 to student model

import keras
import numpy as np
import tensorflow as tf
# To grow the memory usage as is needed by the process
# Get rid of the error: cuDNN failed to initialize

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
from keras.datasets import mnist
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten, Activation
from keras.models import Sequential, Model
from keras import layers
from keras.utils import np_utils
import time
from tensorflow.keras import losses
from tensorflow.keras.utils import to_categorical

**Preprocessing the CIFAR 10 dataset for LeNet5**

In [ ]:
num_classes = 10
(x_train, y_train), (x_test, y_test) =  tf.keras.datasets.cifar10.load_data()

# Enabling One Hot Encoding
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

# Changing input image datatype to float
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalizaig data
x_train  /= 255
x_test /= 255

# **Defining LeNet 5 Model**

In [21]:
model = Sequential()

model.add(Conv2D(6, kernel_size=(5, 5), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(16, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(120, activation='relu'))

model.add(Dense(84, activation='relu'))

# Defining activation function separately here to get the logits/ dark knowledge from teacher model
model.add(Dense(10, name="logits"))
model.add(Activation('softmax'))          

# Run model
# Adam Optimization
# Loss = Cross Entropy Loss
# Epochs = 20
model.compile(loss=keras.metrics.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=128, epochs= 20, verbose=1, validation_data=(x_test, y_test))

# Get Test Accuracy
score = model.evaluate(x_test, y_test)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 62s 1ms/step - loss: 1.8008 - acc: 0.3403 - val_loss: 1.6612 - val_acc: 0.3997
Epoch 2/20
50000/50000 [==============================] - 61s 1ms/step - loss: 1.5282 - acc: 0.4470 - val_loss: 1.4626 - val_acc: 0.4760
Epoch 3/20
50000/50000 [==============================] - 61s 1ms/step - loss: 1.4111 - acc: 0.4916 - val_loss: 1.3672 - val_acc: 0.5058
Epoch 4/20
50000/50000 [==============================] - 61s 1ms/step - loss: 1.3336 - acc: 0.5239 - val_loss: 1.3084 - val_acc: 0.5348
Epoch 5/20
50000/50000 [==============================] - 61s 1ms/step - loss: 1.2671 - acc: 0.5482 - val_loss: 1.2561 - val_acc: 0.5508
Epoch 6/20
50000/50000 [==============================] - 61s 1ms/step - loss: 1.2070 - acc: 0.5682 - val_loss: 1.2299 - val_acc: 0.5639
Epoch 7/20
50000/50000 [==============================] - 60s 1ms/step - loss: 1.1611 - acc: 0.5879 - val_loss: 1.2094 - val_acc

In [ ]:
getSoftmaxKnowledge = Model(inputs=model.input, outputs=model.get_layer("logits").output)
model_logits = getSoftmaxKnowledge.predict(x_train)

In [ ]:
# Defining function described by Geoffrey Hinton in his paper of Knowledge Distillation
def softmax_with_temperature(logits, temperature):
    logits = logits / temperature
    return (np.exp(logits) / np.sum(np.exp(logits)))

In [ ]:
# Temperature is a hyperparameter
temperature = 5
softened_train_prob = softmax_with_temperature(model_logits, temperature)

# **Defining Student Model**

In [ ]:
# Model Definition for the Student Model
def build_small_model():
    inputs = layers.Input(shape = (3072,))
    
    x = layers.Dense(50, activation='relu', name='FC1')(inputs)
    
    x = layers.Dense(10, name='logits')(x)
    
    preds = layers.Activation('softmax', name='Softmax')(x)
  
    model = Model(inputs=inputs, outputs=preds)
    model.summary()
    return model

In [69]:
small_model = build_small_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 3072)              0         
_________________________________________________________________
FC1 (Dense)                  (None, 50)                153650    
_________________________________________________________________
logits (Dense)               (None, 10)                510       
_________________________________________________________________
Softmax (Activation)         (None, 10)                0         
Total params: 154,160
Trainable params: 154,160
Non-trainable params: 0
_________________________________________________________________


In [72]:
(x_train, y_train), (x_test, y_test) =  tf.keras.datasets.cifar10.load_data()

# Enabling One Hot Encoding
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

# Changing input image datatype to float
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalizaig data
x_train  /= 255
x_test /= 255

x_train = x_train.reshape([50000, 3072])
x_test = x_test.reshape([10000, 3072])
    
    
# Optimization = N Adam
# Loss = Cross Entropy loss
# Epochs = 50
# Trained with dark knowledge
small_model.compile(optimizer='Nadam', loss= 'categorical_crossentropy', metrics=['categorical_accuracy'])
small_model.fit(x_train, softened_train_prob, epochs=50, batch_size=128)

test_loss, test_acc = small_model.evaluate(x_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

Epoch 1/50
50000/50000 [==============================] - 9s 181us/step - loss: 4.3646e-05 - categorical_accuracy: 0.4250
Epoch 2/50
50000/50000 [==============================] - 7s 137us/step - loss: 4.3605e-05 - categorical_accuracy: 0.4324
Epoch 3/50
50000/50000 [==============================] - 7s 132us/step - loss: 4.3589e-05 - categorical_accuracy: 0.4355
Epoch 4/50
50000/50000 [==============================] - 7s 133us/step - loss: 4.3584e-05 - categorical_accuracy: 0.4338
Epoch 5/50
50000/50000 [==============================] - 7s 137us/step - loss: 4.3565e-05 - categorical_accuracy: 0.4393
Epoch 6/50
50000/50000 [==============================] - 7s 137us/step - loss: 4.3541e-05 - categorical_accuracy: 0.4404
Epoch 7/50
50000/50000 [==============================] - 6s 130us/step - loss: 4.3528e-05 - categorical_accuracy: 0.4442
Epoch 8/50
50000/50000 [==============================] - 7s 130us/step - loss: 4.3522e-05 - categorical_accuracy: 0.4448
Epoch 9/50
50000/50000 [

In [ ]:
small_model.save('model_50_CIFAR10_5.h5')
model.save('model_LeNet5_CIFAR10.h5')